In [1]:
!nvidia-smi | grep "Tesla\|MiB"

|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |


In [2]:
import subprocess
weights = subprocess.getoutput('ls | grep "weights"')
print(weights)

weights-improvement-09-1.1340-bigger.hdf5


## First input file

Las inquietudes de Shanti Andia.

In [3]:
filename = 'santia_andia.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
banned_dict = {"\n": " ", "á": "a", "é": "e", "í": "i", "ó": "o", "ú": "u", "#":" ", "-":" "}
banned_dict
# raw_text.replace()

{'\n': ' ',
 'á': 'a',
 'é': 'e',
 'í': 'i',
 'ó': 'o',
 'ú': 'u',
 '#': ' ',
 '-': ' '}

In [4]:
def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

multipleReplace(raw_text[1100:1220], banned_dict)

'iii.  el capitan zaldumbide iv.  de otras personas distinguidas que formaban la tripulacion de «el dragon» v.  los dos t'

In [5]:

# Load Larger LSTM network and generate text
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical
# load ascii text and covert to lowercase
filename = '/content/santia_andia.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = multipleReplace(raw_text.lower(), banned_dict)
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
# filename = weights
# model.load_weights(filename)


model.compile(loss='categorical_crossentropy', optimizer='adam')

# model.load_weights('/content/weights-improvement-20-1.5095-bigger.hdf5')
# define the checkpoint
# filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
# checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
# callbacks_list = [checkpoint]
# fit the model
# model.fit(X, y, epochs=1, batch_size=512, callbacks=callbacks_list)
model.fit(X, y, epochs=10, batch_size=512)



Total Characters:  490177
Total Vocab:  59
Total Patterns:  490077
Epoch 1/10
958/958 [==============================] - 91s 86ms/step - loss: 2.7562
Epoch 2/10
958/958 [==============================] - 86s 89ms/step - loss: 2.4850
Epoch 3/10
958/958 [==============================] - 87s 91ms/step - loss: 2.3349
Epoch 4/10
958/958 [==============================] - 87s 91ms/step - loss: 2.2218
Epoch 5/10
958/958 [==============================] - 87s 91ms/step - loss: 2.1335
Epoch 6/10
958/958 [==============================] - 87s 91ms/step - loss: 2.0664
Epoch 7/10
958/958 [==============================] - 87s 91ms/step - loss: 2.0131
Epoch 8/10
958/958 [==============================] - 87s 91ms/step - loss: 1.9691
Epoch 9/10
958/958 [==============================] - 87s 91ms/step - loss: 1.9311
Epoch 10/10
958/958 [==============================] - 87s 91ms/step - loss: 1.8977


In [7]:
# Has the model learnt anything?

In [6]:
import keras

def generate_text(output_len: int, model: keras.engine.sequential.Sequential):
  start = np.random.randint(0, len(dataX)-1)
  pattern = dataX[start]
  understandable_seed = ''.join([int_to_char[value] for value in pattern])
  print("Seed:")
  print("\"", understandable_seed, "\"")
  print(f"You are generating an output text with len {output_len} chars")
  # print(" END SEED")
  t_p = list()
  print("Resulting Sentence")
  # generate characters
  for i in range(output_len):
    x = np.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = np.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    # sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
    t_p.append(result)
  # print("\nDone.")
  understandable_result = "".join(t_p)
  # print(result)
  print("\n")
  print(f"Resulting sentence: {understandable_result}")
  # print("\nDone.")
  # return understandable_result


generate_text(1000, model)

Seed:
"  proximas al acantilado, se veian a la luz de la luna.  frayburu seguia en su desolacion y en su tri "
You are generating an output text with len 1000 chars
Resulting Sentence


Resulting sentence: ste de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la cara de la car

In [7]:
print(" The model has not learnt much :( ")

 The model has not learnt much :( 


In [ ]:
import numpy as np
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical



filename = '/content/santia_andia.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = multipleReplace(raw_text.lower(), banned_dict)
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)

Total Characters:  490177
Total Vocab:  59
Total Patterns:  490077


In [8]:
from keras.optimizers import Adam
import sys
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import to_categorical


model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
opt = Adam(learning_rate=0.0000125)

filename = weights
model.load_weights(filename)


model.compile(loss='categorical_crossentropy', optimizer=opt)


# define the checkpoint
filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=10, batch_size=64, callbacks=callbacks_list)

# pick a random seed
start = np.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
  x = np.reshape(pattern, (1, len(pattern), 1))
  x = x / float(n_vocab)
  prediction = model.predict(x, verbose=0)
  index = np.argmax(prediction)
  result = int_to_char[index]
  seq_in = [int_to_char[value] for value in pattern]
  sys.stdout.write(result)
  pattern.append(index)
  pattern = pattern[1:len(pattern)]
print("\nDone.")

Epoch 1/10
7655/7658 [============================>.] - ETA: 0s - loss: 1.1342
Epoch 1: loss improved from inf to 1.13413, saving model to weights-improvement-01-1.1341-bigger.hdf5
7658/7658 [==============================] - 142s 18ms/step - loss: 1.1341
Epoch 2/10
7656/7658 [============================>.] - ETA: 0s - loss: 1.1347
Epoch 2: loss did not improve from 1.13413
7658/7658 [==============================] - 141s 18ms/step - loss: 1.1347
Epoch 3/10
7658/7658 [==============================] - ETA: 0s - loss: 1.1347
Epoch 3: loss did not improve from 1.13413
7658/7658 [==============================] - 142s 19ms/step - loss: 1.1347
Epoch 4/10
7658/7658 [==============================] - ETA: 0s - loss: 1.1354
Epoch 4: loss did not improve from 1.13413
7658/7658 [==============================] - 141s 18ms/step - loss: 1.1354
Epoch 5/10
7656/7658 [============================>.] - ETA: 0s - loss: 1.1346
Epoch 5: loss did not improve from 1.13413
7658/7658 [====================

In [9]:
generate_text(300, model)

Seed:
" n _el dragon._  allen era un buen muchacho, pero muy poco marino. por mas que yo intente explicarle  "
You are generating an output text with len 300 chars
Resulting Sentence


Resulting sentence: con el capitan y de la casa de la casa de la casa de la casa de la carretera.    ¿y que ha contestado?    no,    ¿pues este?    no.    ¿pue pasa?    si.    ¿pueses que haber?    no, no. no le parece marida.    ¿que puesesa   dijo dilos a iacer la cara de la cara de la cala     en la casa de la casa 


In [10]:
generate_text(300, model)

Seed:
" alizar en zalamerias.  cuando me dejaban entrar en la sala, me pasaba el tiempo mirandolos y diciend "
You are generating an output text with len 300 chars
Resulting Sentence


Resulting sentence: o que el capitan era un hombre de aguirre, el capitan sandow era un capitan en la casa de la casa de la casa de la cala de la calle de la carreta.    ¿que te pasaba?  le dije yo  . es pue esta con el capitan sandow que se ha la cara de caples a la calle de la cara de la cara.    ¿tabes a que se le p


In [18]:
def is_overfit(input_seq_test: str, corpus: str = raw_text, chars_ahead: int = 150):
  i_0 = corpus.find(input_seq_test)
  i_f = i_0 + chars_ahead
  print(corpus[i_0:i_f])

q = "yo no se si estara bien..., porque las g"
is_overfit(q)

yo no se si estara bien..., porque las gentes diran que ...    eso ya os lo he dicho antes. la muchacha esta en ese estado. ya lo sabemos. conque reso


In [12]:
q = "quiza le remordian sus crimenes.  antes de ser n"
is_overfit(q)

quiza le remordian sus crimenes.  antes de ser negrero, el viejo, segun decian, habia hecho naufragar varios barcos asegurados, llegando hasta exponer


In [14]:
generate_text(300, model)

Seed:
" iria nada, sino que si veia algun espia en la finca lo zambulliria en el estanque.  salimos de alla; "
You are generating an output text with len 300 chars
Resulting Sentence


Resulting sentence:  en el camino de la casa era un caritan de la calle de la calle de la carretera y de la carretera y la de las animas.  el capitan se habia conteduido de nada, me dijo que era una manera deserperada y me dijo:    no habra si la seña estan el mar.  el capitan se habia conteduo de su padre, el capitan 


In [15]:
generate_text(300, model)

Seed:
" en el cementerio de la aldea, y tendidos entre dos sepulcros, resguardados del viento, pudimos desca "
You are generating an output text with len 300 chars
Resulting Sentence


Resulting sentence: rsar en la cabeza.  el capitan no era un hombre de aguirre, el capitan sandow era un capitan de la casa de la casa de la casa de la carretera.  el capitan no era un hombre de aguirre, el capitan sandow era un capitan de la casa de la casa de la casa de la carretera.  el capitan no era un hombre de a


In [22]:
q = "resguardados del viento, pudimos desca"
is_overfit(q)

resguardados del viento, pudimos descansar y dormir.  a media noche nos despertamos de hambre y de frio. nos levantamos, salimos del cementerio y echa


In [43]:
# What happens when we retrain on new data?

In [57]:
import subprocess
weights = subprocess.getoutput('ls | grep "weights"')
print(weights)

weights-improvement-01-1.1439-bigger.hdf5
weights-improvement-03-1.1425-bigger.hdf5
weights-improvement-04-1.1423-bigger.hdf5
weights-improvement-15-1.1389-bigger.hdf5


In [59]:
generate_text(300, model)

Seed:
" ierta, proxima a un pueblecito que tenia su puerto.  yo habia oido decir que en algunos puntos de es "
You are generating an output text with len 300 chars
Resulting Sentence


Resulting sentence: cribir el camino de la casa de la casa en el camino de la calara del capitan y la carrasa en la cabeza.  el capitan no era un hombre de aguirre, el capitan sandow era un capitan con un cartillo de proa, y estaba alguna de las casas de la carreta.  el capitan no era un hombre de aguirre, el capitan s


In [60]:
q = "yo habia oido decir que en algunos puntos de es"
is_overfit(q)

yo habia oido decir que en algunos puntos de escocia y de irlanda comen esas algas que se llaman laminarias, y era tal nuestra hambre, que intentamos 


In [26]:
def clean_new_file(text: str, previous_chars: dict) -> str:
  text_ = "".join([char for char in text if char in previous_chars.keys()])
  return text_

In [44]:
print(" we need the new corpus to have the same characters as the previous one")

 we need the new corpus to have the same characters as the previous one


In [28]:
# load ascii text and covert to lowercase
filename = '/content/filosofia_fundamental'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = multipleReplace(raw_text.lower(), banned_dict)
raw_text = clean_new_file(raw_text, char_to_int)
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
 seq_in = raw_text[i:i + seq_length]
 seq_out = raw_text[i + seq_length]
 dataX.append([char_to_int[char] for char in seq_in])
 dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = np.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
# load the network weights
filename = "weights-improvement-06-1.1332-bigger.hdf5"
model.load_weights(filename)

opt = Adam(learning_rate=0.001225)

model.compile(loss='categorical_crossentropy', optimizer=opt)

filepath = "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
model.fit(X, y, epochs=10, batch_size=512, callbacks=callbacks_list)



Total Characters:  501821
Total Vocab:  59
Total Patterns:  501721
Epoch 1/10
980/980 [==============================] - ETA: 0s - loss: 1.7831
Epoch 1: loss improved from inf to 1.78312, saving model to weights-improvement-01-1.7831-bigger.hdf5
980/980 [==============================] - 91s 90ms/step - loss: 1.7831
Epoch 2/10
980/980 [==============================] - ETA: 0s - loss: 1.5405
Epoch 2: loss improved from 1.78312 to 1.54050, saving model to weights-improvement-02-1.5405-bigger.hdf5
980/980 [==============================] - 89s 90ms/step - loss: 1.5405
Epoch 3/10
980/980 [==============================] - ETA: 0s - loss: 1.4590
Epoch 3: loss improved from 1.54050 to 1.45901, saving model to weights-improvement-03-1.4590-bigger.hdf5
980/980 [==============================] - 89s 90ms/step - loss: 1.4590
Epoch 4/10
980/980 [==============================] - ETA: 0s - loss: 1.4084
Epoch 4: loss improved from 1.45901 to 1.40836, saving model to weights-improvement-04-1.4084-b

In [30]:
generate_text(300, model)

Seed:
" udes determinadas, y estas en cierta relacion fija, que nosotros calculamos, comparandola con una ex "
You are generating an output text with len 300 chars
Resulting Sentence


Resulting sentence: tension en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espaci


In [42]:
generate_text(300, model)

Seed:
" pesado eso que llamamos realidad. estoy en un mundo donde yo mando, quiero: y el coche esta pronto,  "
You are generating an output text with len 300 chars
Resulting Sentence


Resulting sentence: es el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el espacio en el 


In [41]:
print("Despite the loss metric has improved Model has lost its touch")

Despite the loss metric has improved Model has lost its touch
